En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

-----
#### Enfoque y lógica soluciones *q_time*

Para las soluciones *q_time* del challenge, el enfoque se centra en optimizar el tiempo de ejecución. Utilizado la biblioteca **Polars**, que es una biblioteca de manipulación de datos de alto rendimiento y baja memoria se realizó la lectura y procesamiento de los datos.

Polars utiliza una estructura de datos llamada LazyFrame que es similar a la de Pandas, pero utiliza un enfoque de evaluación perezosa (lazy evaluation), lo que significa que las operaciones no se ejecutan inmediatamente, sino que se almacenan en un plan de ejecución y sólo se realizan cuando se necesita el resultado. Esto puede llevar a un uso más eficiente de los recursos y a un mejor rendimiento.

**Mejoras al enfoque:** 
- Se podría utilizar servicios de almacenamiento en la nube como *Google Cloud Storage* o *Amazon S3* para almacenar el conjunto de datos. De esta forma la carga de los datos no pasan por la memoria de una sola máquina.

- Podríamos utilizar servicios de Big Data y análisis como *Google BigQuery* o *Amazon Redshift* para procesar el conjunto de datos. Estos servicios están diseñados para trabajar con grandes conjuntos de datos y pueden realizar operaciones de agrupamiento y conteo de manera muy rápida.

En general si se elige un enfoque utilizando herramientas en la nube podriamos aprovechar el conjunto de herramientas que ofrecen las diferentes suits como GCP, AWS, Azure y que interactuan de manera muy eficiente entre si.

#### Enfoque y lógica soluciones *q_memory*

Para las soluciones *q_memory* del challenge el enfoque se centra en optimizar el uso de la memoria. En lugar de cargar todos los datos en la memoria a la vez, se leen los datos en trozos (chunks) de 1000 filas a la vez utilizando la función *read_json* de **Pandas** con el parámetro *chunksize*.

En la inspección previa del archivo pudimos ver que son más de 117000 lineas de tweets, entonces utilizaremos chunks de tamaño 1000 para tener un balance entre un buen tiempo de ejecución y bajo consumo de memoría.

En cuanto al filtrado y procesamiento de los datos utilizamos las clases *Counter* y *defaultdict* de la biblioteca **Collections**, que proporcionan una forma eficiente de contar elementos.

Este enfoque es más eficiente en términos de uso de memoria, ya que sólo mantiene en memoria los contadores y el chunk actual de datos. Sin embargo, puede ser más lento que el enfoque q1_time, especialmente para conjuntos de datos muy grandes, ya que tiene que leer los datos en múltiples pasos.

**Mejoras al enfoque:**

- El uso de herramientas como **Apache Spark** en la nube son especialmente utiles para el procesamiento paralelo de grandes conjuntos de datos. Por ejemplo *Google Cloud Dataproc* (de GCP) o *Amazon EMR* de (AWS) son servicios permiten ejecutar trabajos de Apache Spark en la nube.

- *Paralelización:* Podríamos mejorar el rendimiento de este enfoque utilizando procesamiento paralelo o concurrente. Por ejemplo, podríamos procesar varios chunks de datos simultáneamente en diferentes hilos o procesos.

- Ajuste del tamaño del chunk: El tamaño del chunk seteado a 1000 fue elegido arbitrariamente. Podríamos experimentar con diferentes tamaños de chunk para encontrar el óptimo entre el tiempo de ejecución y el uso de memoria.


----
**Imports**

In [3]:
file_path = "../data/farmers-protest-tweets-2021-2-4.json"

%load_ext memory_profiler

#Funciones desarrolladas
from q1_time import q1_time, q1_time_pandas
from q1_memory import q1_memory

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


----
### Pregunta nro 1

1. Listar las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.


##### **q1_time**

Para optimizar el tiempo de ejecución, cargamos el archivo completo en un *LazyFrame* de **Polars** y utilizamos los métodos de esta libreria ya que son eficientes para el procesamiento de datos ordenados.

El desglose de lo que realizó la función es:
- Seleccionar las columnas relevantes.
- Agrupamos los datos por por fecha y usuario.
- Contar el número de tweets para cada combinación de fecha y usuario.
- Seleccionar las 10 fechas con más tweets.
- Luego iteramos en estas fechas y seleccionar los usuarios con la mayor cantidad de tweets en esos días.
- Se contruye una lista de tuplas con la respuesta.

In [2]:
resultado_q1_time = q1_time(file_path)
display(resultado_q1_time)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Medimos el tiempo que tardó nuestra función en encontrar el resultado y el peak máximo de memoría que utilizó

In [3]:
%timeit q1_time(file_path)
%memit q1_time(file_path)

2.83 s ± 87.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 1677.14 MiB, increment: 522.91 MiB


Con este enfoque podemos ver que la función **q1_time** tiene un tiempo de ejecución promedio de *2.8 segundos* y uso máximo de memoría de *1677 MiB*.

##### **q1_memory**

Para esta solución se leen los datos en chunks y aprovechando las iteraciones para cada chunk, actualiza dos contadores: uno para el conteo de tweets por fecha y otro para el conteo de tweets por usuario para cada fecha. Estos contadores se implementan utilizando las clases *Counter* y *defaultdict* de la biblioteca **Collections**, que proporcionan una forma eficiente de contar elementos.

El desglose de lo que realizó la función es:
- Definir el tamaño del chunk y los contadores.
- Leer el archivo JSON en chunks.
- Actualizar los contadores.
- Obtener las 10 fechas con más tweets.
- Seleccionar el usuario con más tweets para cada fecha.
- Construir una lista de tuplas con la respuesta.

In [4]:
resultado_q1_memory = q1_memory(file_path)
display(resultado_q1_memory)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Medimos el tiempo que tardó nuestra función en encontrar el resultado y el peak máximo de memoría que utilizó

In [3]:
%timeit q1_memory(file_path)
%memit q1_memory(file_path)

4.42 s ± 175 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 157.44 MiB, increment: 15.60 MiB


Con este enfoque podemos ver que la función **q1_time** tiene un tiempo de ejecución promedio de *4.4 segundos* y un uso máximo de memoría de *157 MiB*.

##### **q1_time_pandas**

Como extra mencionar que durante el desarrollo del challenge se intentó utilizar la librería pandas para los enfoques *time* enfoque que no cuida el uso de memoría, pero esta librería tardaba mucho tiempo en leer el conjunto de datos. Por lo que se barajó utilizar la librería **Polars** con su metodo LazyFrame, que es es más rápido que un DataFrame de Pandas gracias a su enfoque de procesamiento perezoso (lazy) que optimiza las operaciones y su capacidad de computación en paralelo aprovechando múltiples núcleos de CPU.

In [6]:
resultado_q1_time_pandas= q1_time_pandas(file_path)
display(resultado_q1_time_pandas)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Medimos el tiempo que tardó nuestra función en encontrar el resultado y el peak máximo de memoría que utilizó

In [4]:
%timeit q1_time_pandas(file_path)
%memit q1_time_pandas(file_path)

5.11 s ± 583 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 2805.34 MiB, increment: 2319.15 MiB


Podemos ver que el uso de memoria y el tiempo de computo utilizando Pandas es mayor que utilizando la librería Polars. 5.11 segundos y 2805 MiB de memoría

----
### Estructura del proyecto


Se siguió la misma estructura definida en el challenge, agregando una carpeta donde se almacenaron los datos

```css
├── README.md
├── requirements.txt
├── LICENSE
├── data
│   └── farmers-protest-tweets-2021-2-4.json
└── src
    ├── challenge.ipynb
    ├── q1_time.py
    ├── q1_memory.py
    ├── q2_time.py
    ├── q2_memory.py
    ├── q3_time.py
    ├── q3_memory.py
    └── utils.py
   ```